In [6]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
groq_api_key = os.environ.get("GROQ_API_KEY")

In [7]:
def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() 
    return text

def split_text(text, chunk_size=1000, chunk_overlap=200):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

In [8]:
def create_vector_store(text_chunks, embeddings):
    return FAISS.from_texts(texts=text_chunks, embedding=embeddings)

In [9]:
def main():
    pdf_path = "1. dietary supplements - for whom.pdf"
    pdf_path = "INDIAN DIETARY SUPPLEMENT MARKET.pdf"
    pdf_path = "2.Hassan2020_Chapter_DietarySupplementsTypesHealthB.pdf"

    if not os.path.exists(pdf_path):
        print(f"Error: File not found at {pdf_path}")
        return

    pdf_text = read_pdf(pdf_path)
    print("Successfully read PDF")

    text_chunks = split_text(pdf_text)
    print(f"Split text into {len(text_chunks)} chunks")

    embeddings = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': False}
    )

    print("Creating vector store...")
    vectorstore = create_vector_store(text_chunks, embeddings)
    print("Vector store created successfully")

    retriever = vectorstore.as_retriever()
    llm = ChatGroq(
        temperature=0.7,
        model_name="llama3-8b-8192",
        groq_api_key=groq_api_key
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=False
    )

    while True:
        query = input("\nEnter your question (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break
        try:
            print("\nProcessing...")
            result = qa_chain.invoke({"query": query})
            print("Query:", query)
            print("\nAnswer:", result["result"])
        except Exception as e:
            print(f"Error processing query: {str(e)}")

if __name__ == "__main__":
    main()

Successfully read PDF
Split text into 61 chunks
Creating vector store...
Vector store created successfully

Processing...
Query: what is the results of dietary supplements and cardiovasular diseases?

Answer: The text does not explicitly state the results of dietary supplements on cardiovascular diseases. However, it does mention a study from 1971 that linked fish oil and omega-3 fatty acids to a lower risk of ischemic heart disease in Eskimos (Greenlanders) who consumed a traditional diet rich in polyunsaturated omega-3 fatty acids.

Processing...
Query: what is the results of Omega-3 fatty acids?

Answer: According to the provided context, the results of Omega-3 fatty acids include:

* Lower levels of plasma total lipids, plasma cholesterol, plasma triglycerides, and pre β-lipoprotein (very low-density lipoprotein) in Greenlanders eating a traditional meat and fish diet rich in polyunsaturated omega-3 fatty acids compared to Danes and Greenlanders living in Denmark (Bang et al. 1971)